In [ ]:
import numpy as np, math, re
from itertools import product

def read_file_lines(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        
        return [line.strip() for line in lines]

In [ ]:
test_input = [
    "0:", "###", "##.", "##.", "",
    "1:", "###", "##.", ".##", "",
    "2:", ".##", "###", "##.", "",
    "3:", "##.", "###", "##.", "",
    "4:", "###", "#..", "###", "",
    "5:", "###", ".#.", "###", "",
    "4x4: 0 0 0 0 2 0",
    "12x5: 1 0 1 0 2 2",
    "12x5: 1 0 1 0 3 2"
]

In [ ]:
def parse_input(text):
    if isinstance(text, list):
        text = "\n".join(text)

    shapes = []
    regions = []

    lines = iter(text.strip().splitlines())
    for raw in lines:
        line = raw.strip()
        if not line:
            continue

        if re.match(r'^\d+:\s*$', line):
            shape = []
            for _ in range(3):
                row = next(lines).strip()
                shape.append([1 if c == "#" else 0 for c in row])
            shapes.append(shape)

        elif ":" in line and not re.match(r'^\d+:\s*$', line):
            key, values = line.split(":")
            dims = [int(x) for x in key.strip().lower().split("x")]
            nums = [int(x) for x in values.strip().split()]
            regions.append([dims, nums])

    return shapes, regions

from copy import deepcopy

def rotate_90(arr):
    return [list(row) for row in zip(*arr[::-1])]

def flip_vertical(arr):
    return arr[::-1]

def flip_horizontal(arr):
    return [row[::-1] for row in arr]

def generate_orientations(shape):
    """Generate all unique rotations/flips of a shape."""
    orientations = []

    # original + 3 rotations
    current = shape
    for _ in range(4):
        orientations.append(current)
        current = rotate_90(current)

    # horizontal flip + rotations
    flipped_h = flip_horizontal(shape)
    current = flipped_h
    for _ in range(4):
        orientations.append(current)
        current = rotate_90(current)

    # vertical flip + rotations
    flipped_v = flip_vertical(shape)
    current = flipped_v
    for _ in range(4):
        orientations.append(current)
        current = rotate_90(current)

    # remove duplicates
    unique_orientations = []
    seen = set()
    for o in orientations:
        t = tuple(tuple(row) for row in o)
        if t not in seen:
            seen.add(t)
            unique_orientations.append(o)

    return unique_orientations

def place_shape_on_grid(grid, shape):
    """
    Place a single shape on every position + orientation on a grid.
    Returns a list of new grids where no cell exceeds 1.
    """
    rows = len(grid)
    cols = len(grid[0])
    placements = []

    orientations = generate_orientations(shape)

    for rot in orientations:
        s_rows = len(rot)
        s_cols = len(rot[0])

        for r in range(rows - s_rows + 1):
            for c in range(cols - s_cols + 1):
                new_grid = deepcopy(grid)
                overlap = False
                for i in range(s_rows):
                    for j in range(s_cols):
                        new_grid[r+i][c+j] += rot[i][j]
                        if new_grid[r+i][c+j] > 1:
                            overlap = True
                            break
                    if overlap:
                        break
                if not overlap:
                    placements.append(new_grid)

    return placements

def place_shapes_recursively(grid, list_of_shapes):
    """
    Recursively place each shape in list_of_shapes on the previous placement grids.
    Skip any grid that would cause an overlap (cell > 1).
    """
    if not list_of_shapes:
        return [grid]  # base case: no shapes left

    first_shape = list_of_shapes[0]
    rest_shapes = list_of_shapes[1:]

    # place the first shape on current grid
    first_placements = place_shape_on_grid(grid, first_shape)

    # recursively place remaining shapes on each valid placement
    all_grids = []
    for g in first_placements:
        all_grids.extend(place_shapes_recursively(g, rest_shapes))

    return all_grids
    
def process_regions(input):
    s, regions = parse_input(input)

    count = 0

    for region in regions:
        grid = [[0 for _ in range(region[0][0])] for _ in range(region[0][1])]
        r = region[1]

        list_of_shapes = []
        for i in range(0, len(r)):
            for j in range(0, r[i]):
                list_of_shapes.append(s[i])

        grids = place_shapes_recursively(grid, list_of_shapes)
            
        if len(grids) > 0:
            count += 1
    
    return count

# process_regions(test_input)
process_regions(read_file_lines('day12_input.txt'))

In [ ]:
from copy import deepcopy

def parse_input(text):
    if isinstance(text, list):
        text = "\n".join(text)

    shapes = []
    regions = []

    lines = iter(text.strip().splitlines())
    for raw in lines:
        line = raw.strip()
        if not line:
            continue

        if re.match(r'^\d+:\s*$', line):
            shape = []
            for _ in range(3):
                row = next(lines).strip()
                shape.append([1 if c == "#" else 0 for c in row])
            shapes.append(shape)

        elif ":" in line and not re.match(r'^\d+:\s*$', line):
            key, values = line.split(":")
            dims = [int(x) for x in key.strip().lower().split("x")]
            nums = [int(x) for x in values.strip().split()]
            regions.append([dims, nums])

    return shapes, regions

class Shape:
    def __init__(self, base_grid):
        # Pre-compute all 4 rotations like Rust version
        self.rotations = []
        
        # Rotation 0: original
        self.rotations.append(base_grid)
        
        # Rotation 1: 90 degrees
        rot1 = [[0]*3 for _ in range(3)]
        for i in range(3):
            for j in range(3):
                rot1[j][2-i] = base_grid[i][j]
        self.rotations.append(rot1)
        
        # Rotation 2: 180 degrees
        rot2 = [[0]*3 for _ in range(3)]
        for i in range(3):
            for j in range(3):
                rot2[2-i][2-j] = base_grid[i][j]
        self.rotations.append(rot2)
        
        # Rotation 3: 270 degrees
        rot3 = [[0]*3 for _ in range(3)]
        for i in range(3):
            for j in range(3):
                rot3[2-j][i] = base_grid[i][j]
        self.rotations.append(rot3)
        
        # Count cells for feasibility check
        self.cell_count = sum(sum(row) for row in base_grid)

def can_place_shape(grid, shape, x, y, rotate):
    """Check if shape can be placed at position (x,y) with given rotation."""
    m = len(grid)
    n = len(grid[0])
    
    if x + 3 > m or y + 3 > n:
        return False
    
    rotated = shape.rotations[rotate]
    
    for i in range(3):
        for j in range(3):
            if rotated[i][j] == 1:
                if grid[x + i][y + j] > 0:
                    return False
    return True

def place_shape(grid, shape, x, y, rotate, remove=False):
    """Place or remove shape at position (x,y) with given rotation."""
    rotated = shape.rotations[rotate]
    
    for i in range(3):
        for j in range(3):
            if rotated[i][j] == 1:
                if remove:
                    grid[x + i][y + j] -= 1
                else:
                    grid[x + i][y + j] += 1

def check_feasibility(shapes, grid, needed):
    """Check if remaining shapes can fit in empty cells."""
    m = len(grid)
    n = len(grid[0])
    
    # Count empty cells
    empty_cells = sum(1 for row in grid for cell in row if cell == 0)
    
    # Count cells needed by remaining shapes
    needed_cells = sum(needed[i] * shapes[i].cell_count for i in range(len(shapes)))
    
    return needed_cells <= empty_cells

def solve_recursive(shapes, grid, needed, depth=0):
    """Recursive backtracking solver with pruning."""
    m = len(grid)
    n = len(grid[0])
    
    # Early pruning: check if remaining shapes can fit
    if not check_feasibility(shapes, grid, needed):
        return 0
    
    # Check if all shapes are placed
    if all(count == 0 for count in needed):
        return 1
    
    # Try placing each available shape
    for shape_idx in range(len(shapes)):
        if needed[shape_idx] == 0:
            continue
            
        # Try all positions and rotations
        for x in range(m):
            for y in range(n):
                for rotate in range(4):
                    if can_place_shape(grid, shapes[shape_idx], x, y, rotate):
                        # Place shape
                        place_shape(grid, shapes[shape_idx], x, y, rotate, remove=False)
                        needed[shape_idx] -= 1
                        
                        # Recurse
                        result = solve_recursive(shapes, grid, needed, depth + 1)
                        
                        if result == 1:
                            return 1
                        
                        # Backtrack
                        place_shape(grid, shapes[shape_idx], x, y, rotate, remove=True)
                        needed[shape_idx] += 1
    
    return 0

def process_regions(input_data):
    """Process all regions and count solvable ones."""
    shape_grids, regions = parse_input(input_data)
    
    # Convert to Shape objects with pre-computed rotations
    shapes = [Shape(grid) for grid in shape_grids]
    
    count = 0
    
    for region in regions:
        m, n = region[0]
        needed = region[1]
        
        # Create empty grid
        grid = [[0] * m for _ in range(n)]
        
        # Solve with backtracking
        if solve_recursive(shapes, grid, needed[:]) == 1:
            count += 1
    
    return count

# process_regions(test_input)
process_regions(read_file_lines('day12_input.txt'))